In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Lemmatize edilmiş CSV dosyasını yükle
df = pd.read_csv('./data_without_extra_features/lemmatized_and_misspelled_removed_SEFACED.csv')

# Veriyi özellikler (lemmatize edilmiş metin) ve hedef değişken olarak ayır
X = df['lemmatized_tokens']
y = df['Class_Label']

# Etiketleri sayısal değerlere dönüştür
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Metinleri tokenle
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Maksimum dizi uzunluğunu tanımla
maxlen = 100

# Dizileri aynı uzunlukta olacak şekilde doldur
X_pad = pad_sequences(X_seq, maxlen=maxlen)

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# LSTM modelini tanımla
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Modeli derle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğit
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Modeli değerlendir

y_pred_prob = model.predict(X_test)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)

print(classification_report(y_test, y_pred))


Epoch 1/10
200/200 [==============================] - 51s 214ms/step - loss: 0.2475 - accuracy: 0.8839 - val_loss: 0.0816 - val_accuracy: 0.9588
Epoch 2/10
200/200 [==============================] - 41s 205ms/step - loss: 0.0547 - accuracy: 0.9759 - val_loss: 0.0675 - val_accuracy: 0.9706
Epoch 3/10
200/200 [==============================] - 41s 207ms/step - loss: 0.0340 - accuracy: 0.9858 - val_loss: 0.0809 - val_accuracy: 0.9712
Epoch 4/10
200/200 [==============================] - 41s 204ms/step - loss: 0.0277 - accuracy: 0.9878 - val_loss: 0.0786 - val_accuracy: 0.9725
Epoch 5/10
200/200 [==============================] - 42s 210ms/step - loss: 0.0246 - accuracy: 0.9883 - val_loss: 0.0875 - val_accuracy: 0.9719
Epoch 6/10
200/200 [==============================] - 42s 210ms/step - loss: 0.0244 - accuracy: 0.9889 - val_loss: 0.0958 - val_accuracy: 0.9712
Epoch 7/10
200/200 [==============================] - 42s 211ms/step - loss: 0.0243 - accuracy: 0.9887 - val_loss: 0.1030 - val_ac